In [1]:
import pandas as pd
import utils
import plotly.graph_objects as go

In [2]:
pair = "EUR_USD"
granularity = "H1"
ma_list = [8,16,32,64,128,256]

In [3]:
df = pd.read_pickle(utils.get_hist_data_filename(pair, granularity))

In [4]:
df.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2022-01-19T02:00:00.000000000Z,1800,1.13225,1.13280,1.13201,1.13226,1.13217,1.13274,1.13194,1.13219,1.13233,1.13288,1.13208,1.13232
1,2022-01-19T03:00:00.000000000Z,1396,1.13224,1.13300,1.13211,1.13276,1.13216,1.13294,1.13204,1.13268,1.13231,1.13307,1.13218,1.13283
2,2022-01-19T04:00:00.000000000Z,1142,1.13278,1.13316,1.13257,1.13302,1.13270,1.13310,1.13250,1.13294,1.13285,1.13323,1.13264,1.13310
3,2022-01-19T05:00:00.000000000Z,1116,1.13304,1.13336,1.13285,1.13300,1.13297,1.13329,1.13278,1.13292,1.13310,1.13344,1.13292,1.13307
4,2022-01-19T06:00:00.000000000Z,1447,1.13302,1.13366,1.13270,1.13343,1.13294,1.13358,1.13263,1.13336,1.13309,1.13374,1.13277,1.13350


In [5]:
non_cols = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 0 to 3998
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   time    3999 non-null   object 
 1   volume  3999 non-null   int64  
 2   mid_o   3999 non-null   float64
 3   mid_h   3999 non-null   float64
 4   mid_l   3999 non-null   float64
 5   mid_c   3999 non-null   float64
 6   bid_o   3999 non-null   float64
 7   bid_h   3999 non-null   float64
 8   bid_l   3999 non-null   float64
 9   bid_c   3999 non-null   float64
 10  ask_o   3999 non-null   float64
 11  ask_h   3999 non-null   float64
 12  ask_l   3999 non-null   float64
 13  ask_c   3999 non-null   float64
dtypes: float64(12), int64(1), object(1)
memory usage: 437.5+ KB


In [6]:
#get the last 100 candles
df_plot = df.iloc[-100:]

In [7]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x = df_plot.time, open = df_plot.mid_o, high = df_plot.mid_h, low = df_plot.mid_l, close = df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor="#23A06B",
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color="#2EC886",
    decreasing_line_color="#FF3A4C"
))
fig.update_layout(width=1000,height=400, font = dict(size=10, color = "#e1e1e1"), paper_bgcolor="#1e1e1e", plot_bgcolor="#1e1e1e")
fig.update_xaxes(gridcolor = "#1f292f", showgrid=True, fixedrange=True, rangeslider=dict(visible=False))
fig.update_yaxes(gridcolor= "#1f292f", showgrid=True)
fig.show()

In [8]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [9]:
for ma in ma_list:
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()

In [12]:
df_ma.dropna(inplace=True)
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_8,MA_16,MA_32,MA_64,MA_128,MA_256
255,2022-02-02T17:00:00.000000000Z,1.12994,1.13121,1.12984,1.13113,1.130921,1.129399,1.127805,1.124312,1.121442,1.126909
256,2022-02-02T18:00:00.000000000Z,1.13112,1.13144,1.13058,1.13093,1.130928,1.129596,1.127953,1.124546,1.121464,1.126904
257,2022-02-02T19:00:00.000000000Z,1.13092,1.13123,1.13061,1.13103,1.130854,1.129843,1.128068,1.124779,1.121488,1.126897
258,2022-02-02T20:00:00.000000000Z,1.13102,1.13140,1.13038,1.13055,1.130613,1.130036,1.128206,1.125004,1.121511,1.126888
259,2022-02-02T21:00:00.000000000Z,1.13054,1.13083,1.13022,1.13055,1.130539,1.130238,1.128330,1.125223,1.121529,1.126878


In [13]:
df_plot = df_ma.iloc[-200:].copy()
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x = df_plot.time, open = df_plot.mid_o, high = df_plot.mid_h, low = df_plot.mid_l, close = df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor="#23A06B",
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color="#2EC886",
    decreasing_line_color="#FF3A4C"
))
for ma in ma_list:
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time, y=df_plot[col], line = dict(width = 2), line_shape='spline', name=col))
fig.update_layout(width=1000,height=400, font = dict(size=10, color = "#e1e1e1"), paper_bgcolor="#1e1e1e", plot_bgcolor="#1e1e1e")
fig.update_xaxes(gridcolor = "#1f292f", showgrid=True, fixedrange=True, rangeslider=dict(visible=False))
fig.update_yaxes(gridcolor= "#1f292f", showgrid=True)
fig.show()